# Feature Engineering

---

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import* 
import pandas as pd
import geopandas as gpd

In [3]:
spark = (
    SparkSession.builder.appName("Feature_Engineering")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.driver.memory","4G")
    .config("spark.executor.memory","4G")
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/09 11:19:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/09 11:19:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/09 11:19:51 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/09/09 11:19:51 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


# Read datasets

In [4]:
consumer_detail  = spark.read.parquet('../data/curated/consumer_detail')
transaction = spark.read.parquet('../data/curated/transaction')
merchant = spark.read.parquet('../data/curated/merchant')
postcode_sa2_geo = gpd.read_file('../data/curated/postcode_sa2_geo.shp')

### Convert data types

In [7]:
print('consumer_detail:')
consumer_detail.printSchema()

print('\ntransaction:')
transaction.printSchema()

print('\nmerchant:')
merchant.printSchema()

print('\npostcode_sa2_geo:')
print(postcode_sa2_geo.dtypes)

consumer_detail:
root
 |-- consumer_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- gender: string (nullable = true)


transaction:
root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)


merchant:
root
 |-- merchant_abn: long (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- fraud_probability: double (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- revenue_level: string (nullable = true)
 |-- take_rate: float (nullable = true)


postcode_sa2_geo:
SA2_name        object
SA2_MAINCO       int64
postcode         int64
state           object
geometry      geometry
dtype: object


In [6]:
# convert the data type of postcode column in consumer_transaction dataset 
consumer_detail = consumer_detail.withColumn('postcode', col('postcode').cast('int'))
consumer_detail.printSchema()

root
 |-- consumer_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- gender: string (nullable = true)



### Display dataset

In [22]:
consumer_detail.show(5)

+-----------+-------+-----------------+-----+--------+------+
|consumer_id|user_id|             name|state|postcode|gender|
+-----------+-------+-----------------+-----+--------+------+
|     870353| 213579|    Charles Davis|   SA|    5261|  Male|
|     923963| 213580|Jacqueline Nelson|  QLD|    4744|Female|
|      93016| 213581|    Carolyn Smith|  QLD|    4454|Female|
|      61324| 213582|      Denise Rush|   WA|    6705|Female|
|     823311| 213583|  Nathan Williams|  NSW|    2145|  Male|
+-----------+-------+-----------------+-----+--------+------+
only showing top 5 rows



In [23]:
consumer_detail.count()

499999

In [24]:
consumer_detail.show(5)

+-----------+-------+-----------------+-----+--------+------+
|consumer_id|user_id|             name|state|postcode|gender|
+-----------+-------+-----------------+-----+--------+------+
|     870353| 213579|    Charles Davis|   SA|    5261|  Male|
|     923963| 213580|Jacqueline Nelson|  QLD|    4744|Female|
|      93016| 213581|    Carolyn Smith|  QLD|    4454|Female|
|      61324| 213582|      Denise Rush|   WA|    6705|Female|
|     823311| 213583|  Nathan Williams|  NSW|    2145|  Male|
+-----------+-------+-----------------+-----+--------+------+
only showing top 5 rows



In [25]:
transaction.show(5)

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  14935| 79417999332|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|
|      1| 46451548968| 72.61581642788431|76bab304-fa2d-400...|    2021-11-26|
|  14936| 89518629617|3.0783487174439297|a2ae446a-2959-41c...|    2021-11-26|
|      1| 49167531725| 51.58228625503599|7080c274-17f7-4cc...|    2021-11-26|
|  14936| 31101120643|25.228114942417797|8e301c0f-06ab-45c...|    2021-11-26|
+-------+------------+------------------+--------------------+--------------+
only showing top 5 rows



In [26]:
transaction.count()

13614675

# Compute total transaction count and value for each merchant

In [9]:
# group by merchant_abn and aggregate the count of orders and total dollar_value
transaction_detail = transaction.groupBy("merchant_abn").agg(
    count("order_id").alias("transaction_count"),  # assuming tip count is equivalent to the number of orders
    sum("dollar_value").alias("total_dollar_value")
)

transaction_detail.show(5)

+------------+-----------------+------------------+
|merchant_abn|transaction_count|total_dollar_value|
+------------+-----------------+------------------+
| 83412691377|            14288| 498536.5816973135|
| 38700038932|             7132| 9546185.360697312|
| 35344855546|             1522|134737.25046268434|
| 15613631617|             1785| 543030.5313328261|
| 19839532017|              726|          113982.0|
+------------+-----------------+------------------+
only showing top 5 rows



In [28]:
# save the parquet file
transaction_detail.write.parquet('../data/curated/transactions_count_dollar', mode='overwrite')

# Merge consumer and transaction by `user_id`

In [10]:
consumer_transaction = consumer_detail.join(
    transaction,
    on=['user_id'],
    how='inner'
)

consumer_transaction = consumer_detail.join(
    transaction,
    on=['user_id'],
    how='inner'
)

consumer_transaction.show(5)

+-------+-----------+----------------+-----+--------+------+------------+------------------+--------------------+--------------+
|user_id|consumer_id|            name|state|postcode|gender|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+-----------+----------------+-----+--------+------+------------+------------------+--------------------+--------------+
|  14935|    1059280|   Cameron Adams|  QLD|    4563|  Male| 79417999332|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|
|      1|    1195503|Yolanda Williams|   WA|    6935|Female| 46451548968| 72.61581642788431|76bab304-fa2d-400...|    2021-11-26|
|  14936|     986886|     Maria Riley|   SA|    5157|Female| 89518629617|3.0783487174439297|a2ae446a-2959-41c...|    2021-11-26|
|      1|    1195503|Yolanda Williams|   WA|    6935|Female| 49167531725| 51.58228625503599|7080c274-17f7-4cc...|    2021-11-26|
|  14936|     986886|     Maria Riley|   SA|    5157|Female| 31101120643|25.228114942417797|8e301

In [11]:
consumer_transaction.count()

13614675

In [31]:
# save the parquet file
consumer_transaction.write.parquet('../data/curated/consumer_transaction', mode='overwrite')

# Compute total transaction count and value for each consumer

In [32]:
consumer_transaction.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- consumer_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [12]:
# calculate the total number of transactions and total transaction amount for each consumer
consumer_total_count_dollar = (consumer_transaction
                               .groupBy("consumer_id")
                               .agg(
                                   count("order_id").alias("total_transaction_count"),
                                   sum("dollar_value").alias("total_dollar_value")
                                   ))

extra_columns_df = (consumer_transaction
                    .select("consumer_id", "user_id", "postcode")
                    .distinct())

consumer_total_count_dollar = (consumer_total_count_dollar
                               .join(extra_columns_df, on="consumer_id", how="left"))

consumer_total_count_dollar.show(5)


+-----------+-----------------------+------------------+-------+--------+
|consumer_id|total_transaction_count|total_dollar_value|user_id|postcode|
+-----------+-----------------------+------------------+-------+--------+
|    1144223|                    543| 77840.35704704828|  14971|    4184|
|     283969|                    581|100112.93516413833|  15024|    2228|
|    1050479|                    564|100977.04005104418|  15031|    5502|
|    1174371|                    562| 87222.61085702278|     95|    2352|
|     921339|                    591|134278.40943370626|    152|    2161|
+-----------+-----------------------+------------------+-------+--------+
only showing top 5 rows



In [13]:
# save as a parquet file
consumer_total_count_dollar.write.parquet('../data/curated/consumer_total_count_dollar', mode='overwrite')

# Add geometry information to previous table

In [69]:
consumer_total_count_dollar_df = consumer_total_count_dollar.toPandas()
consumer_total_count_dollar_geo = consumer_total_count_dollar_df.merge(
    postcode_sa2_geo,
    on='postcode', how='inner')
consumer_total_count_dollar_geo = consumer_total_count_dollar_geo.drop('SA2_MAINCO', axis=1)
consumer_total_count_dollar_geo.head()

,consumer_id,total_transaction_count,total_dollar_value,user_id,postcode,SA2_name,state,geometry
0,1144223,543,77840.357047,14971,4184,North Stradbroke Island,QLD,"MULTIPOLYGON (((153.3841 -27.51581, 153.38428 ..."
1,1144223,543,77840.357047,14971,4184,Southern Moreton Bay Islands,QLD,"MULTIPOLYGON (((153.31943 -27.68148, 153.31947..."
2,242288,558,87245.973644,20696,4184,North Stradbroke Island,QLD,"MULTIPOLYGON (((153.3841 -27.51581, 153.38428 ..."
3,242288,558,87245.973644,20696,4184,Southern Moreton Bay Islands,QLD,"MULTIPOLYGON (((153.31943 -27.68148, 153.31947..."
4,453040,586,83765.845889,3783,4184,North Stradbroke Island,QLD,"MULTIPOLYGON (((153.3841 -27.51581, 153.38428 ..."


In [70]:
consumer_total_count_dollar_geo.shape

(37159, 8)

In [71]:
consumer_total_count_dollar_bySA2_geo = consumer_total_count_dollar_geo.groupby('SA2_name', as_index=False).agg({
    'total_transaction_count': 'sum',
    'total_dollar_value': 'sum',
    'postcode': 'first',                # keep the first postcode of each SA2_name
    'state': 'first',
    'geometry': 'first'     
})
consumer_total_count_dollar_bySA2_geo.head()

,SA2_name,total_transaction_count,total_dollar_value,postcode,state,geometry
0,ACT - South West,13006,2.002136e+06,2902,ACT,"POLYGON ((148.88381 -35.26411, 148.94988 -35.2..."
1,APY Lands,9080,1.400538e+06,872,NT,"POLYGON ((129.00186 -26.72252, 129.00186 -26.7..."
2,Abbotsford,3369,5.278622e+05,3067,VIC,"POLYGON ((144.99255 -37.80249, 144.99266 -37.8..."
3,Aberfoyle Park,3959,6.247266e+05,5159,SA,"POLYGON ((138.58963 -35.06584, 138.58993 -35.0..."
4,Acacia Gardens,4526,7.369201e+05,2763,NSW,"POLYGON ((150.91593 -33.72971, 150.91661 -33.7..."


In [72]:
consumer_total_count_dollar_bySA2_geo.shape

(2311, 6)

In [75]:
# save the DataFrame as a CSV file
consumer_total_count_dollar_bySA2_geo.to_csv('../data/curated/consumer_total_count_dollar_bySA2_geo.csv', index=False)

In [74]:
# drop the geometry column
consumer_total_count_dollar_bySA2_df = consumer_total_count_dollar_bySA2_geo.drop(columns=['geometry'])

# convert the DataFrame as a Parquet file
consumer_total_count_dollar_bySA2 = spark.createDataFrame(consumer_total_count_dollar_bySA2_df)

# save the DataFrame as a Parquet file
consumer_total_count_dollar_bySA2.write.parquet('../data/curated/consumer_total_count_dollar_bySA2', mode='overwrite')